In [3]:
import os
import re
import aocd
import numpy as np
import pandas as pd
from aocd import submit
from tqdm import tqdm
from collections import Counter


hex_2_bin = """0 = 0000
1 = 0001
2 = 0010
3 = 0011
4 = 0100
5 = 0101
6 = 0110
7 = 0111
8 = 1000
9 = 1001
A = 1010
B = 1011
C = 1100
D = 1101
E = 1110
F = 1111""".split('\n')
hex_2_bin = {x.split(' = ')[0]:x.split(' = ')[1] for x in hex_2_bin}
test = """620080001611562C8802118E34"""

data = """020D74FCE27E600A78020200DC298F1070401C8EF1F21A4D6394F9F48F4C1C00E3003500C74602F0080B1720298C400B7002540095003DC00F601B98806351003D004F66011148039450025C00B2007024717AFB5FBC11A7E73AF60F660094E5793A4E811C0123CECED79104ECED791380069D2522B96A53A81286B18263F75A300526246F60094A6651429ADB3B0068937BCF31A009ADB4C289C9C66526014CB33CB81CB3649B849911803B2EB1327F3CFC60094B01CBB4B80351E66E26B2DD0530070401C82D182080803D1C627C330004320C43789C40192D002F93566A9AFE5967372B378001F525DDDCF0C010A00D440010E84D10A2D0803D1761045C9EA9D9802FE00ACF1448844E9C30078723101912594FEE9C9A548D57A5B8B04012F6002092845284D3301A8951C8C008973D30046136001B705A79BD400B9ECCFD30E3004E62BD56B004E465D911C8CBB2258B06009D802C00087C628C71C4001088C113E27C6B10064C01E86F042181002131EE26C5D20043E34C798246009E80293F9E530052A4910A7E87240195CC7C6340129A967EF9352CFDF0802059210972C977094281007664E206CD57292201349AA4943554D91C9CCBADB80232C6927DE5E92D7A10463005A4657D4597002BC9AF51A24A54B7B33A73E2CE005CBFB3B4A30052801F69DB4B08F3B6961024AD4B43E6B319AA020020F15E4B46E40282CCDBF8CA56802600084C788CB088401A8911C20ECC436C2401CED0048325CC7A7F8CAA912AC72B7024007F24B1F789C0F9EC8810090D801AB8803D11E34C3B00043E27C6989B2C52A01348E24B53531291C4FF4884C9C2C10401B8C9D2D875A0072E6FB75E92AC205CA0154CE7398FB0053DAC3F43295519C9AE080250E657410600BC9EAD9CA56001BF3CEF07A5194C013E00542462332DA4295680"""
#data = test
data, hex_2_bin

('020D74FCE27E600A78020200DC298F1070401C8EF1F21A4D6394F9F48F4C1C00E3003500C74602F0080B1720298C400B7002540095003DC00F601B98806351003D004F66011148039450025C00B2007024717AFB5FBC11A7E73AF60F660094E5793A4E811C0123CECED79104ECED791380069D2522B96A53A81286B18263F75A300526246F60094A6651429ADB3B0068937BCF31A009ADB4C289C9C66526014CB33CB81CB3649B849911803B2EB1327F3CFC60094B01CBB4B80351E66E26B2DD0530070401C82D182080803D1C627C330004320C43789C40192D002F93566A9AFE5967372B378001F525DDDCF0C010A00D440010E84D10A2D0803D1761045C9EA9D9802FE00ACF1448844E9C30078723101912594FEE9C9A548D57A5B8B04012F6002092845284D3301A8951C8C008973D30046136001B705A79BD400B9ECCFD30E3004E62BD56B004E465D911C8CBB2258B06009D802C00087C628C71C4001088C113E27C6B10064C01E86F042181002131EE26C5D20043E34C798246009E80293F9E530052A4910A7E87240195CC7C6340129A967EF9352CFDF0802059210972C977094281007664E206CD57292201349AA4943554D91C9CCBADB80232C6927DE5E92D7A10463005A4657D4597002BC9AF51A24A54B7B33A73E2CE005CBFB3B4A30052801F69DB4B08F3B6961024AD4B43E6

In [5]:
b = '' 
for x in data:
    b += hex_2_bin[x]
#b = '00111000000000000110111101000101001010010001001000000000'
def split_into_parts(binary):
    v = binary[:3]
    t = binary[3:6]
    #print('binary',binary)
    if t == '100':
        
        subpackets = ''
        # literal value
        #print('literal',v,t)
        s, e = 6,11
        subpack = binary[s:e]
        #print(subpack,subpack[0] == '1')
        while subpack[0] == '1':
            
            subpackets += subpack[1:]
            s,e = s+5,e+5
            subpack = binary[s:e]
            #print(subpack)
        subpackets += subpack[1:]
        rest = binary[e:]
        #print('subs',subpackets)
        #print('rest',rest)
        out = {'type':'literal','v':int(v,2),'t':t,'val':int(subpackets,2)}
    else:
        #operator
        subpackets = []
        i = int(binary[6])
        #print('operator',v,t,i)
        if i == 0:
            #print('i',i)
            l = binary[7:22]

            length = int(l,2)
            #print('length',l,length)
            #not yet split
            subpackets += [binary[22:22+length]]
            #print(subpackets)
            rest=binary[22+length:]
            #print(rest)
            #print('subs',subpackets)
            #print('rest',rest)
            print(length)
            out = {'type':'operator','v':int(v,2),'t':t,'i':i,'l':l,'sub':split_into_parts(subpackets[0])}
        else:
            #print('i',i)
            l = binary[7:18]
            length = int(l,2)
            for j in range(length):
                subpackets += [binary[18:]]
            rest = ''
            #print('length',l,length)
            #print('subs',subpackets)
            #print('rest',rest)
            print(length)
            out = {'type':'operator','v':int(v,2),'t':t,'i':i,'l':l,'sub':split_into_parts(subpackets[0])}

    if (len(rest) > 0) and (int(rest)!=0):
        #print('looking at rest')
        return [out] + split_into_parts(rest)
    else:
        return [out]
        


a = split_into_parts(b)
def sum_versions(a):
    #print(a['v'],a['sub'],isinstance(a['sub'],list))
    if a.get('sub',None) is not None:
        return a['v'] + sum([sum_versions(sub) for sub in a['sub']])
    else:
        return a['v']

sum_versions(a[0]),a

53
2
2
3
3
1
1
398
376
354
332
1
1
1
1
1
220
198
1
158
136
114
1
1
56
2
72
105
2
2
104
5
3
2
3
3
3
3
33
100
47
31
2
106
33
3
2
2
120
75
32
1
2
27
1
2
2
2
2
33
33
3
3
33
33
2
2
101
2
64
3
4
2
2
87
1
2
62
32
160
2
33
3
115
72
63
144
106
3
33
4
110
57
5
2
2
27
2
42


(893,
 [{'type': 'operator',
   'v': 0,
   't': '000',
   'i': 1,
   'l': '00000110101',
   'sub': [{'type': 'literal', 'v': 6, 't': '100', 'val': 62231},
    {'type': 'operator',
     'v': 7,
     't': '001',
     'i': 1,
     'l': '00000000010',
     'sub': [{'type': 'operator',
       'v': 4,
       't': '111',
       'i': 1,
       'l': '00000000010',
       'sub': [{'type': 'operator',
         'v': 0,
         't': '000',
         'i': 1,
         'l': '00000000011',
         'sub': [{'type': 'literal', 'v': 3, 't': '100', 'val': 5},
          {'type': 'literal', 'v': 1, 't': '100', 'val': 15},
          {'type': 'literal', 'v': 0, 't': '100', 'val': 3},
          {'type': 'operator',
           'v': 4,
           't': '000',
           'i': 1,
           'l': '00000000011',
           'sub': [{'type': 'literal', 'v': 4, 't': '100', 'val': 14},
            {'type': 'literal', 'v': 7, 't': '100', 'val': 15},
            {'type': 'literal', 'v': 4, 't': '100', 'val': 6},
          

(16,
 [{'type': 'operator',
   'v': 4,
   't': '010',
   'i': 1,
   'l': '00000000001',
   'sub': [{'type': 'operator',
     'v': 1,
     't': '010',
     'i': 1,
     'l': '00000000001',
     'sub': [{'type': 'operator',
       'v': 5,
       't': '010',
       'i': 0,
       'l': '000000000001011',
       'sub': [{'type': 'literal', 'v': 6, 't': '100', 'val': 15}]}]}]}])

In [24]:
def get_adjacent(i,j, data):
    corners = []
    if i != (len(data) - 1):
        corners += [ (i+1,j) ]
    if j != (len(data[i]) - 1):
        corners += [(i,j+1)]
    return corners

def calc_path_risk(path, data):
    tot = sum([data[x] for x in path])
    last = path[-1]
    _max = tot + (len(data)-last[0])*9 + (len(data[0])-last[1])*9 + 1
    _min = tot + (len(data)-last[0])*1 + (len(data[0])-last[1])*1 + 1
    return tot, _min, _max#, tot>_max

ongoing_paths = [(0,0,0,sum(data)-data[0,0])]
i = 0
j = 0
path_no = 0
done_paths = []
k = 0
end_risks = []
while len(ongoing_paths):
    ongoing_paths.sort(key=lambda x: x[2] + x[3])
    ongoing_paths = ongoing_paths[1:100]
    path = ongoing_paths[0]
    print('looking at path',path)
    ongoing_paths = ongoing_paths[1:100]
    i,j,risk, h = path
    options = get_adjacent(i,j,data)
    if len(options) == 0:
        print('Found 1 end, risk:',risk)
        end_risks += [risk]
        done_paths += [path]
        ongoing_paths = ongoing_paths[1:]
        continue
    for option in options:
        new_i,new_j = option
        print(i*j/(len(data)*len(data[0])), len(ongoing_paths))
        new_risk = risk + data[new_i,new_j]
        new_h = np.mean(data[new_i:,new_j:])*(len(data)-new_i) + (len(data[0])-new_j) - data[new_i, new_j]
        ongoing_paths += [(new_i,new_j,new_risk,new_h)]
    #print(ongoing_paths)
    k+=1
    if k == 10000:
        break
    #ongoing_paths = ongoing_paths[1:]
min(end_risks)

looking at path (0, 0, 0, array([21, 26, 49, 43, 51, 28, 35, 42, 37, 40]))
0.0 0
0.0 1
looking at path (1, 0, 1, 43.5)
0.0 1
0.0 2
looking at path (2, 0, 3, 38.4)
0.0 2
0.0 3
looking at path (3, 0, 6, 34.2)
0.0 3
0.0 4
looking at path (4, 0, 13, 24.7)
0.0 4
0.0 5
looking at path (5, 0, 14, 27.2)
0.0 5
0.0 6
looking at path (6, 0, 15, 23.6)
0.0 6
0.0 7
looking at path (7, 0, 18, 17.9)
0.0 7
0.0 8
looking at path (8, 0, 19, 16.3)
0.0 8
0.0 9
looking at path (9, 0, 21, 11.8)
0.0 9
looking at path (9, 1, 24, 10.0)
0.09 9
looking at path (8, 1, 21, 14.777777777777779)
0.08 9
0.08 10
looking at path (9, 1, 24, 10.0)
0.09 10
looking at path (9, 2, 25, 11.125)
0.18 10
looking at path (9, 2, 25, 11.125)
0.18 10
looking at path (9, 3, 26, 10.571428571428571)
0.27 10
looking at path (9, 3, 26, 10.571428571428571)
0.27 10
looking at path (8, 2, 30, 7.125)
0.16 10
0.16 11
looking at path (9, 4, 35, 2.166666666666668)
0.36 11
looking at path (9, 4, 35, 2.166666666666668)
0.36 11
looking at path (7, 

0.36 99
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (4, 2, 22, 25.25)
0.08 99
0.08 100
looking at path (5, 2, 20, 27.25)
0.1 99
0.1 100
looking at path (6, 2, 25, 19.25)
0.12 99
0.12 100
looking at path (7, 2, 27, 18.125)
0.14 99
0.14 100
looking at path (8, 2, 36, 7.125)
0.16 99
0.16 100
looking at path (7, 3, 32, 14.142857142857142)
0.21 99
0.21 100
looking at path (8, 3, 35, 11.857142857142858)
0.24 99
0.24 100
looking at path (9, 3, 36, 10.571428571428571)
0.27 99
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (4, 2, 22, 25.25)
0.08 99
0.08 100
looking at path (6, 2, 28, 19.25)
0.12 99
0.12 100
looking at path (9, 5, 42, 5.4)
0.45 99
looking at path (7, 5, 32, 15.40000

0.32 99
0.32 100
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (8, 5, 39, 10.2)
0.4 99
0.4 100
looking at path (9, 5, 43, 5.4)
0.45 99
looking at path (8, 4, 36, 13.5)
0.32 99
0.32 100
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (8, 5, 39, 10.2)
0.4 99
0.4 100
looking at path (9, 5, 43, 5.4)
0.45 99
looking at path (8, 4, 36, 13.5)
0.32 99
0.32 100
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (8, 5, 39, 10.2)
0.4 99
0.4 100
looking at path (9, 5, 43, 5.4)
0.45 99
looking at path (8, 4, 36, 13.5)
0.32 99
0.32 100
looking at path (9, 4, 45, 2.166666666666668)
0.36 99
looking at path (8, 5, 39, 10.2)
0.4 99
0.4 100
looking at path (9, 5, 43, 5.4)
0.45 99
looking at path (8, 6, 45, 4.5)
0.48 99
0.48 100
looking at path (9, 6, 45, 4.5)
0.54 99
looking at path (8, 6, 45, 4.5)
0.48 99
0.48 100
looking at path (9, 6, 45, 4.5)
0.54 99
looking at path (8, 6, 45, 4.5)
0.48 99
0.48 100
looking at path (9, 6, 45, 4.5)
0.54 9

looking at path (7, 6, 32, 16.25)
0.42 99
0.42 100
looking at path (8, 6, 40, 4.5)
0.48 99
0.48 100
looking at path (7, 7, 38, 10.333333333333336)
0.49 99
0.49 100
looking at path (8, 7, 43, 5.333333333333332)
0.56 99
0.56 100
looking at path (9, 6, 44, 4.5)
0.54 99
looking at path (9, 6, 44, 4.5)
0.54 99
looking at path (9, 7, 47, 2.666666666666667)
0.63 99
looking at path (6, 7, 35, 14.666666666666664)
0.42 99
0.42 100
looking at path (8, 7, 45, 5.333333333333332)
0.56 99
0.56 100
looking at path (8, 7, 45, 5.333333333333332)
0.56 99
0.56 100
looking at path (9, 6, 46, 4.5)
0.54 99
looking at path (6, 8, 37, 13.5)
0.48 99
0.48 100
looking at path (6, 9, 38, 12.0)
0.54 99
looking at path (7, 9, 47, 3.0)
0.63 99
looking at path (8, 9, 48, 2.0)
0.72 99
looking at path (9, 9, 49, 1.0)
Found 1 end, risk: 49
looking at path (9, 7, 48, 2.666666666666667)
0.63 97
looking at path (4, 5, 26, 24.8)
0.2 97
0.2 98
looking at path (5, 5, 28, 21.6)
0.25 98
0.25 99
looking at path (6, 5, 29, 18.4)
0

looking at path (9, 7, 51, 2.666666666666667)
0.63 97
looking at path (9, 7, 51, 2.666666666666667)
0.63 97
looking at path (8, 8, 48, 6.0)
0.64 97
0.64 98
looking at path (8, 9, 49, 2.0)
0.72 98
looking at path (9, 9, 50, 1.0)
Found 1 end, risk: 50
looking at path (8, 8, 48, 6.0)
0.64 96
0.64 97
looking at path (8, 9, 49, 2.0)
0.72 97
looking at path (9, 9, 50, 1.0)
Found 1 end, risk: 50
looking at path (8, 8, 48, 6.0)
0.64 95
0.64 96
looking at path (8, 9, 49, 2.0)
0.72 96
looking at path (9, 9, 50, 1.0)
Found 1 end, risk: 50
looking at path (8, 8, 48, 6.0)
0.64 94
0.64 95
looking at path (8, 9, 49, 2.0)
0.72 95
looking at path (9, 9, 50, 1.0)
Found 1 end, risk: 50
looking at path (8, 7, 49, 5.333333333333332)
0.56 93
0.56 94
looking at path (9, 8, 56, -1.5)
0.72 94
looking at path (9, 8, 56, -1.5)
0.72 94
looking at path (9, 8, 56, -1.5)
0.72 94
looking at path (9, 8, 56, -1.5)
0.72 94
looking at path (9, 7, 52, 2.666666666666667)
0.63 94
looking at path (9, 7, 52, 2.666666666666667

looking at path (8, 9, 55, 2.0)
0.72 98
looking at path (9, 9, 56, 1.0)
Found 1 end, risk: 56
looking at path (9, 8, 59, -1.5)
0.72 96
looking at path (9, 8, 59, -1.5)
0.72 96
looking at path (9, 8, 59, -1.5)
0.72 96
looking at path (9, 8, 59, -1.5)
0.72 96
looking at path (9, 8, 59, -1.5)
0.72 96
looking at path (5, 3, 39, 18.57142857142857)
0.15 96
0.15 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)
0.63 97
looking at path (9, 7, 55, 2.666666666666667)

Found 1 end, risk: 60
looking at path (9, 9, 60, 1.0)
Found 1 end, risk: 60
looking at path (9, 9, 60, 1.0)
Found 1 end, risk: 60
looking at path (9, 9, 60, 1.0)
Found 1 end, risk: 60
looking at path (9, 9, 60, 1.0)
Found 1 end, risk: 60
looking at path (7, 8, 50, 11.0)
0.56 80
0.56 81
looking at path (8, 8, 52, 6.0)
0.64 81
0.64 82
looking at path (8, 9, 53, 2.0)
0.72 82
looking at path (9, 9, 54, 1.0)
Found 1 end, risk: 54
looking at path (7, 8, 50, 11.0)
0.56 80
0.56 81
looking at path (8, 8, 52, 6.0)
0.64 81
0.64 82
looking at path (8, 9, 53, 2.0)
0.72 82
looking at path (9, 9, 54, 1.0)
Found 1 end, risk: 54
looking at path (8, 8, 55, 6.0)
0.64 80
0.64 81
looking at path (8, 9, 56, 2.0)
0.72 81
looking at path (9, 9, 57, 1.0)
Found 1 end, risk: 57
looking at path (8, 8, 55, 6.0)
0.64 79
0.64 80
looking at path (8, 9, 56, 2.0)
0.72 80
looking at path (9, 9, 57, 1.0)
Found 1 end, risk: 57
looking at path (8, 8, 55, 6.0)
0.64 78
0.64 79
looking at path (8, 9, 56, 2.0)
0.72 79
looking 

43

In [27]:
len(done_paths)
min([calc_path_risk(path,data)[0] - data[0,0] for path in done_paths])

40

In [5]:
option

(1, 0)

In [62]:
from copy import deepcopy
#for each pair, we know which new pair will be created
prev, rules = data
rules = rules.split('\n')
rules = [x.split(' -> ') for x in rules]
#rules = {k:v for k,v in rules}

added_counts = []
for x,mid in rules:
    added_counts += [[x, [x[0]+mid,mid+x[1]]]]
added_counts = {k:v for k,v in added_counts}

prev, rules
n_steps = 10
prev_counts = Counter([prev[j:j+2] for j in range(len(prev)-1)])
for i in tqdm(range(1,n_steps+1)):
    print(i,prev_counts,'\n')
    new_counts = deepcopy(prev_counts)
    for c in prev_counts:
        #print('c',c)
        if prev_counts[c] == 0:
            #print('skipping, no values')
            continue
        additions = added_counts[c]
            
        for a in additions:
            #print('a',a)
            if new_counts.get(a,None) is None:
                new_counts[a] = 0
            new_counts[a] += prev_counts[c]
        new_counts[c] -= 1
    prev_counts = new_counts
prev_counts

100%|██████████| 10/10 [00:00<00:00, 12610.66it/s]

1 Counter({'NN': 1, 'NC': 1, 'CB': 1}) 

2 Counter({'NC': 1, 'CN': 1, 'NB': 1, 'BC': 1, 'CH': 1, 'HB': 1, 'NN': 0, 'CB': 0}) 

3 Counter({'CB': 2, 'NB': 2, 'BC': 2, 'BB': 2, 'CN': 1, 'CC': 1, 'BH': 1, 'HC': 1, 'NN': 0, 'NC': 0, 'CH': 0, 'HB': 0}) 

4 Counter({'NB': 5, 'BB': 5, 'BC': 4, 'HB': 3, 'CN': 2, 'CH': 2, 'BN': 2, 'NC': 1, 'CB': 1, 'CC': 1, 'BH': 1, 'HH': 1, 'NN': 0, 'HC': 0}) 

5 Counter({'NB': 15, 'BB': 15, 'BC': 8, 'BN': 8, 'CB': 5, 'CN': 4, 'HB': 3, 'BH': 3, 'HC': 3, 'CH': 2, 'CC': 2, 'NC': 1, 'HH': 1, 'HN': 1, 'NH': 1, 'NN': 0}) 

6 Counter({'NB': 45, 'BB': 45, 'BN': 30, 'BC': 19, 'CN': 10, 'HB': 10, 'CB': 9, 'CH': 7, 'BH': 7, 'HC': 6, 'CC': 5, 'NC': 3, 'HH': 3, 'HN': 1, 'NH': 1, 'NN': 0}) 

7 Counter({'BB': 138, 'NB': 137, 'BN': 104, 'BC': 46, 'CB': 25, 'CN': 25, 'HB': 24, 'BH': 20, 'CH': 16, 'HC': 16, 'CC': 14, 'HH': 9, 'NC': 8, 'HN': 3, 'NH': 3, 'NN': 0}) 

8 Counter({'BB': 424, 'NB': 419, 'BN': 345, 'BC': 115, 'CN': 66, 'CB': 64, 'HB': 64, 'BH': 55, 'CH': 43, 'HC': 42, 

Counter({'NN': 0,
         'NC': 612,
         'CB': 1232,
         'CN': 1400,
         'NB': 12091,
         'BC': 2087,
         'CH': 897,
         'HB': 1230,
         'CC': 781,
         'BB': 12240,
         'BH': 1163,
         'HC': 894,
         'BN': 11047,
         'HH': 652,
         'HN': 351,
         'NH': 351})

In [46]:
counts = Counter([prev[j:j+2] for j in range(len(prev)-1)])
counts['CB'] += 1
counts

Counter({'CB': 2,
         'BN': 1,
         'NB': 1,
         'BO': 2,
         'OK': 1,
         'KH': 1,
         'HV': 1,
         'VB': 1,
         'ON': 1,
         'NC': 1,
         'CP': 1,
         'PP': 1,
         'PB': 1,
         'BB': 1,
         'BC': 1,
         'CK': 1,
         'KV': 1,
         'VH': 1})

In [47]:
rules

for c in counts:
    print(c)

CB
BN
NB
BO
OK
KH
HV
VB
ON
NC
CP
PP
PB
BB
BC
CK
KV
VH
